# Task.

1) **Импортируйте данные data.csv.zip (в папке) с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.**

2) **Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.**

3) ⭐️Задание со звёздочкой!⭐️

**Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
Сколько всего заказов отменили пользователи?**

4) Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

5) 💀 Сложное задание! 💀

**Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id). Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.**

6) ⭐️Задание со звёздочкой!⭐️

**Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.**

7) **Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST?** 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

8) **Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.**

9) **Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)**

Например, для следующего примера...
```
InvoiceNo StockCode Quantity InvoiceDate          UnitPrice  CustomerID  Revenue
536365    85123A    6        2010-12-01 08:26:00  2.55       17850.0     15.30
536365    71053     6        2010-12-01 08:26:00  3.39       17850.0     20.34
555555    71053     1        2010-12-02 18:00:00  3.39       17850.0     3.39
```
...сумма заказа будет равна:
```
InvoiceNo  TotalRevenue
536365     35.64
555555     3.39
```
И ответом будет:
536365, 555555

Подгрузим все нужные библиотеки.

In [81]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Считаем данные, с которыми будем работать, и посмотрим на них.

In [82]:
data = pd.read_csv('data.csv.zip', compression='zip', encoding='ISO-8859-1')

Описание данных и head.

Данные о транзакциях в период с 01.12.2010 по 12.09.2011:

* InvoiceNo – номер транзакции
* StockCode – код товара
* Description – описание товара
* Quantity – количество единиц товара, добавленных в заказ
* InvoiceDate – дата транзакции 
* UnitPrice – цена за единицу товара
* CustomerID – id клиента
* Country – страна, где проживает клиент

In [83]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [84]:
data.info() # Сразу видим типы данных, количество колонок с такими типами и сколько null значений в каждой колонке

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


**Задача 1.**

In [85]:
retail = pd.read_csv('data.csv.zip', compression='zip', encoding='ISO-8859-1')
retail_columns = retail.columns

**Задача 2.**

In [86]:
amount_duplicates = len(retail) - len(retail.drop_duplicates())
amount_duplicates # Количество

5268

In [87]:
retail = retail.drop_duplicates() # Уже без дубликатов

**Задача 3.**

In [88]:
retail.InvoiceNo.apply(lambda x: x[0] == 'C').sum()

9251

**Задача 4.**

In [89]:
retail = retail.query("Quantity > 0")
len(retail)

526054

**Задача 5.**

In [90]:
ret_ger_80_perc = retail.query("Country == 'Germany'").groupby('CustomerID', as_index=False) \
    .agg({'InvoiceNo': pd.Series.nunique}).InvoiceNo.quantile(q=0.8) # Нашли 80 персентиль

In [91]:
germany_top = retail.query("Country == 'Germany'").groupby('CustomerID', as_index=False) \
    .agg({'InvoiceNo': pd.Series.nunique}).query("InvoiceNo > @ret_ger_80_perc").CustomerID # Нашли id
germany_top

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
26    12524.0
37    12569.0
44    12600.0
51    12619.0
52    12621.0
56    12626.0
64    12647.0
69    12662.0
77    12705.0
78    12708.0
79    12709.0
81    12712.0
83    12720.0
Name: CustomerID, dtype: float64

**Задача 6.**

In [92]:
top_retail_germany = retail[retail.CustomerID.isin(germany_top)]
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


**Задача 7.**

In [93]:
top_retail_germany.query("Description != 'POSTAGE'").Description.value_counts().head(1)

ROUND SNACK BOXES SET OF4 WOODLAND     52
Name: Description, dtype: int64

**Задача 8.**

In [96]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


**Задача 9.**

In [95]:
retail.groupby('InvoiceNo', as_index=False).agg({'Revenue': 'sum'}) \
    .sort_values('Revenue', ascending=False).head(5) \
    .rename(columns={'Revenue': 'TotalRevenue'})

,InvoiceNo,TotalRevenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00
